In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
tf.set_random_seed(seed_val)

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def evaluate(final_embedding, word2int, embed_size):
    gensim_w = GensimWrapper('data/news.txt',embed_size, 0, log=False)
    gensim_w.set_embeddings(word2int, final_embedding)
    result = gensim_w.evaluate()
    for key in result:
        print("{0}: {1:.2f}%".format(key, result[key]), end=' ')
    print()

In [3]:
words = read_file()
unkown_word = "<unk>"
# words = [unkown_word] + words
xvocab, xword2int, xint2word = build_vocab(words)

words, word2freq = min_count_threshold(words)
vocab, word2int, int2word = build_vocab(words)
print(len(vocab))
# word2freq = get_frequency(words, word2int, int2word)
unigrams = [word2freq[int2word[i]] for i in range(len(word2int))]

char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 120
embed_size = 100
skip_window = 5

58384


In [4]:
def parseVec(file, delimiter):
    lines = open(file, encoding='utf8').readlines()
    vocab_size, embed_size = [int(s) for s in lines[0].split()]
    vocab_size  = max(len(word2int), vocab_size)
    embeddings = np.ndarray((vocab_size, embed_size), dtype=np.float64)
    for i in range(1, vocab_size):
        try:
            line = lines[i][:-1].split(delimiter)
            word = line[0]
            if word in word2int:
                wordvec = np.array([np.float64(j) for j in line[1:] if j != ''])
                embeddings[word2int[word]] = wordvec
        except Exception as e:
            print(lines[i])
            print(e)
    return embeddings

In [ ]:
util = Utils(word2int, int2word, fast)
print(util.evaluate_word_analogy('data/analogies.txt'))

In [ ]:
utils = Utils(embedding=fast, word2int=word2int, int2word=int2char)
utils.sorted_sim("ኢትዮጵያ")

In [ ]:
evaluate(embed_size=embed_size,final_embedding=normalize(fast), word2int=word2int)
evaluate(embed_size=embed_size,final_embedding=fast, word2int=word2int)

In [ ]:
utils = Utils(embedding=cew, word2int=word2int, int2word=int2char)
utils.sorted_sim("ኢትዮጵያ")

In [9]:
# cew = parseVec('results/w2v.txt', ' ')
# print(cew.shape)
cew = np.load('results/seq_8_100.npy')
# cew = np.concatenate([cew, np.zeros((1, 100))], axis=0)
print(cew.shape)

(297836, 100)


In [10]:
evaluate(embed_size=cew.shape[1],final_embedding=normalize(cew), word2int=xword2int) 
evaluate(embed_size=cew.shape[1],final_embedding=cew, word2int=xword2int)

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


syntactic: 12.15% semantic: 1.11% total: 7.87% pick-one-out: 66.97% 
syntactic: 0.00% semantic: 0.00% total: 0.00% pick-one-out: 33.94% 


In [ ]:
def vec(w, v):
    return v[word2int[w]]

In [ ]:
lines = open('data/analogies.txt', encoding='utf-8').readlines()
e = normalize(cew)
# e = cew
for line in lines:
#     ws = "ልጅቱ ልጁ ወይዘሮ አቶ".split(' ')
    if ":" in line:
        continue
    try:
        ws = line[:-1].split(' ')
        v = normalize(vec(ws[1], e) - vec(ws[0], e) + vec(ws[2], e))
        V = np.dot(e, v)
        sort = list(sorted(enumerate(V[1:]), key=lambda x: x[1], reverse=True))
        so = [int2word[index[0]] for index in sort[:4]]
        if ws[-1] in so:
            print(so, ws[-1])
#         break
    except:
        pass

In [ ]:
lines = open('data/analogies.txt', encoding='utf-8').readlines()
words = []
for line in lines:
    ws = line[:-1].split(' ')
    in_vocab = True
    for w in ws:
        if w not in word2int:
            in_vocab = False
        if in_vocab:
            words.append(ws)
ws = []
for wl in words:
    s = ' '.join(wl)
    ws.append(s)
ws = '\n'.join(ws)
open('data/newan.txt', mode='w', encoding='utf-8').write(ws)

        
        

In [ ]:
lines = open('data/analogies.txt', encoding='utf-8').readlines()
words = []
lns = {}
for line in lines:
    ws = line[:-1].split(' ')
    in_vocab = True
    for w in ws:
        if w not in word2int:
            in_vocab = False
        if in_vocab:
            ln = ' '.join(ws)
            if ln not in lns:
                lns[ln] = len(lns)
            ls = ' '.join(ws[2:4] + ws[0:2])
            if ln not in lns:
                lns[ln] = len(lns)
ws = '\n'.join(lns.keys())
open('data/newan2.txt', mode='w', encoding='utf-8').write(ws)